In [14]:
import numpy as np 
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from PIL import Image

In [39]:
image_dir = "cavallo"

images = []
for filename in os.listdir(image_dir):
    image_path = os.path.join(image_dir, filename)
    image = cv2.imread(image_path)
    
    if image is not None:
        image = cv2.resize(image, (120, 120))
        images.append(image)

print("Number of images read:", len(images))

Number of images read: 2623


In [40]:
def divide_image(image):
    parts = []
    height, width, _ = image.shape
    part_height = height // 3
    part_width = width // 3
    
    for i in range(3):
        for j in range(3):
            part = image[i*part_height:(i+1)*part_height, j*part_width:(j+1)*part_width]
            parts.append(part)
    
    return parts

In [41]:
def generate_combinations(parts, num_combinations):
    combinations = []
    original_positions = []
    indices = list(range(len(parts)))
    
    for _ in range(num_combinations):
        random.shuffle(indices)
        combination = [parts[i] for i in indices]
        combinations.append(combination)
        original_positions.append(indices.copy())
    
    return combinations, original_positions
def stitch_shuffled_image(parts):
    num_parts = len(parts)
    part_size = parts[0].shape[0]  # Assuming all parts are square
    
    stitched_image_size = int(np.sqrt(num_parts) * part_size)
    stitched_image = np.zeros((stitched_image_size, stitched_image_size, parts[0].shape[2]), dtype=np.uint8)
    
    for i in range(stitched_image.shape[0] // part_size):
        for j in range(stitched_image.shape[1] // part_size):
            part_index = i * int(stitched_image.shape[0] / part_size) + j
            stitched_image[i*part_size:(i+1)*part_size, j*part_size:(j+1)*part_size] = parts[part_index]
    
    return stitched_image

In [42]:
input_data = []
target_data = []
non_converted_target_data = []

for image in images:
    parts = divide_image(image)
    combinations, original_positions = generate_combinations(parts, 10)
    
    for idx, combination in enumerate(combinations):
        shuffled_image = stitch_shuffled_image(combination)
        input_data.append(shuffled_image)

        dummy_target = np.zeros((9, 9), dtype=np.uint8)
        for i in range(9):
            dummy_target[i, original_positions[idx][i]] = 1

        target_data.append(dummy_target.flatten())
        non_converted_target_data.append(original_positions[idx])
input_data = np.array(input_data)
target_data = np.array(target_data)

print("Input data shape:", input_data.shape)
print("Target data shape:", target_data.shape)

Input data shape: (26230, 120, 120, 3)
Target data shape: (26230, 81)


In [43]:
import sys
sys.path.append('../vig_pytorch')
from vig import Grapher
num_patches = (120 // 10) * (120 // 10)

grapher_module = Grapher(
    in_channels=4,         # RGB image has 3 channels
    kernel_size=3,         # A common choice, could be different based on your architecture
    dilation=1,            # Standard dilation
    conv='edge',           # Replace 'edge' with the actual type used in your model
    act='relu',            # A common activation function
    norm=None,             # Depends on whether you want to use normalization
    bias=True,             # Typically, biases are used
    stochastic=False,      # If stochastic depth is not used
    epsilon=0.0,           # Hyperparameter for the edge convolution
    r=1,                   # Downsampling rate
    n=num_patches,         # Number of nodes
    drop_path=0.0,         # Drop path rate for stochastic depth
    relative_pos=True,
    # groups = 1     # Set to True if the model uses relative positions
)

c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: 'Could not find module 'C:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


using relative_pos


In [78]:
#grapher_module = grapher_module.to(shuffled_image.device)
# Pass the preprocessed image through the grapher module
# This should give you the output with the graph convolutions applied
#graph_output = grapher_module(images[0])

import numpy as np
import torch
from torchvision import transforms 

def transform_image_for_grapher(image):

    if image.dtype != np.uint8:
        raise ValueError("Expected image dtype to be np.uint8")
    
    # Check input dimensions
    if image.ndim != 3 or image.shape[2] != 3:
        raise ValueError("Image must have three channels (H, W, 3)")

    # Convert to PyTorch tensor
    image_tensor = torch.from_numpy(image)

    # Permute to move the channel to the first dimension (C, H, W)
    image_tensor = image_tensor.permute(2, 0, 1)  # Now shape is [3, 120, 120]

    # Add an extra channel (e.g., an alpha channel filled with ones)
    alpha_channel = torch.ones((1, image.shape[0], image.shape[1]), dtype=image_tensor.dtype)
    image_tensor = torch.cat((image_tensor, alpha_channel), dim=0)  # Now shape is [4, 120, 120]
    # Add a batch dimension
    image_tensor_wbatch = image_tensor.unsqueeze(0) 
    final = image_tensor_wbatch.float()
     # Now shape is [1, 4, 120, 120]
    return final
 # Should print torch.Size([1, 4, 120, 120])
w = transform_image_for_grapher(images[0])
w.shape
graph_output = grapher_module(w)
pool = nn.AdaptiveAvgPool2d((12, 12))
pooled_output = pool(graph_output)


hi


In [79]:
sys.path.append('../vig_pytorch/gcn_lib')
from gcn_lib import DenseDilatedKnnGraph

# Initialize the DenseDilatedKnnGraph module
# Adjust the parameters as needed based on your specific requirements
dense_dilated_knn_graph = DenseDilatedKnnGraph(k=9, dilation=1, stochastic=False, epsilon=0.0)

# Assuming graph_output is your Grapher module output of shape [1, 4, 224, 224]
# Reshape or process graph_output as required to fit the expected input shape of DenseDilatedKnnGraph
# The expected shape might be [batch_size, num_dims, num_points, 1], adjust accordingly

# If you need to flatten or reshape graph_output, ensure it matches the expected dimensions
# For example, if we flatten the spatial dimensions into a single dimension of points:
num_points = pooled_output.shape[2] * pooled_output.shape[3]  # For a 224x224 image, this would be 50176
pooled_output_reshaped = pooled_output.reshape(1, 4, num_points, 1)  # Reshape to [1, 4, 50176, 1]

# Use the module to find edges
edge_index = dense_dilated_knn_graph(pooled_output_reshaped)

In [83]:
import torch
# Let's say your edge_index is a tensor of shape [2, 1, N, k]
# where N is the number of nodes, and k is the number of nearest neighbors
# Flatten the edge_index to work with it more easily
edge_index_flat = edge_index.view(2, -1)  # This will have shape [2, N*k]
# Now, let's create an adjacency matrix of size [N, N]
# We initialize it to zeros,  assuming no connections
edge_index_flat.shape
k=9
N = edge_index_flat.shape[1] // k  # total number of nodes
print(N)
adjacency_matrix = torch.zeros((N, N))
for i in range(edge_index_flat.shape[1]):
    source_node = edge_index_flat[0, i]
    target_node = edge_index_flat[1, i]

    # As edge_index contains indices starting from 0 to N-1
    adjacency_matrix[source_node, target_node] = 1

# If you want an edge list instead, you can simply use the edge_index_flat
edge_list = edge_index_flat.t().tolist() 

num_nodes = 144 #12*12
feature_dimension = 4
graph_features = pooled_output.reshape(1, feature_dimension, num_nodes)

graph_features = graph_features.squeeze(0)  

# If you need to map these features to each node based on your edge list
node_features = {node: [] for node in range(num_nodes)}  # Dictionary to store features for each node

# Assuming 'edge_list' is a list of tuples (target_node_index, source_node_index)
for i in edge_list:
    # Add the source node features to the target node's feature list
    node_features[i[1]].append(graph_features[:, i[0]])
node_features

144


{0: [tensor([ 69.3000, 145.3500, 141.4400,   1.0000], grad_fn=<SelectBackward0>),
  tensor([ 73.9200, 159.0800, 155.4400,   1.0000], grad_fn=<SelectBackward0>),
  tensor([ 83.5600, 176.9800, 175.5200,   1.0000], grad_fn=<SelectBackward0>),
  tensor([ 75.5600, 163.4800, 160.1700,   1.0000], grad_fn=<SelectBackward0>),
  tensor([ 73.1500, 159.3200, 155.8100,   1.0000], grad_fn=<SelectBackward0>),
  tensor([ 72.8500, 159.9100, 155.3800,   1.0000], grad_fn=<SelectBackward0>),
  tensor([ 84.3900, 177.8300, 177.7200,   1.0000], grad_fn=<SelectBackward0>),
  tensor([ 77.6800, 169.1100, 166.4000,   1.0000], grad_fn=<SelectBackward0>),
  tensor([ 80.0600, 173.5300, 172.0500,   1.0000], grad_fn=<SelectBackward0>)],
 1: [tensor([ 82.4800, 158.4300, 156.2300,   1.0000], grad_fn=<SelectBackward0>),
  tensor([ 86.2700, 163.6800, 162.5700,   1.0000], grad_fn=<SelectBackward0>),
  tensor([ 83.0200, 158.6500, 158.4800,   1.0000], grad_fn=<SelectBackward0>),
  tensor([ 87.1000, 161.8900, 163.7200,   1.0

In [87]:
import torch
import torch.nn as nn
import torch.nn.functional as F

def get_node_features_and_edge_list(input_image, grapher_module):
    # Transform input image for the grapher
    w = transform_image_for_grapher(input_image)
    
    # Get graph output from the grapher module
    graph_output = grapher_module(w)
    
    # Perform adaptive average pooling
    pool = nn.AdaptiveAvgPool2d((12, 12))
    pooled_output = pool(graph_output)
    
    # Reshape pooled output for processing
    num_points = 12 * 12
    feature_dimension = pooled_output.shape[1]
    graph_features = pooled_output.view(1, feature_dimension, num_points)
    graph_features = graph_features.squeeze(0)
    
    # Create DenseDilatedKnnGraph object
    dense_dilated_knn_graph = DenseDilatedKnnGraph(k=9, dilation=1, stochastic=False, epsilon=0.0)
    
    # Reshape graph features to fit the expected input shape of DenseDilatedKnnGraph
    graph_features_reshaped = graph_features.view(1, feature_dimension, num_points, 1)
    
    # Use the module to find edges
    edge_index = dense_dilated_knn_graph(graph_features_reshaped)
    
    # Flatten edge_index to work with it more easily
    edge_index_flat = edge_index.view(2, -1)
    
    # Create adjacency matrix
    N = edge_index_flat.shape[1] // 9  # total number of nodes
    # adjacency_matrix = torch.zeros((N, N))
    # for i in range(edge_index_flat.shape[1]):
    #     source_node = edge_index_flat[0, i]
    #     target_node = edge_index_flat[1, i]
    #     adjacency_matrix[source_node, target_node] = 1
    
    # Create edge list
    edge_list = edge_index_flat.t().tolist()
    
    # Create node features
    node_features = {node: [] for node in range(num_points)}
    for i in edge_list:
        node_features[i[1]].append(graph_features[:, i[0]])
    
    return node_features, edge_list
get_node_features_and_edge_list(images[1], grapher_module)

hi


({0: [tensor([101.1900, 123.7900, 111.6400,   1.0000], grad_fn=<SelectBackward0>),
   tensor([127.6700, 153.8600, 136.6000,   1.0000], grad_fn=<SelectBackward0>),
   tensor([ 84.5500, 102.2700,  90.2300,   1.0000], grad_fn=<SelectBackward0>),
   tensor([ 86.3500, 102.1200,  91.3300,   1.0000], grad_fn=<SelectBackward0>),
   tensor([ 82.1300, 102.5200,  88.2800,   1.0000], grad_fn=<SelectBackward0>),
   tensor([ 98.4500, 125.6100, 114.0200,   1.0000], grad_fn=<SelectBackward0>),
   tensor([125.8800, 145.4000, 135.0600,   1.0000], grad_fn=<SelectBackward0>),
   tensor([64.7100, 84.1600, 74.1200,  1.0000], grad_fn=<SelectBackward0>),
   tensor([101.6700, 131.4900, 119.4800,   1.0000], grad_fn=<SelectBackward0>)],
  1: [tensor([ 78.6100, 110.9100,  97.3300,   1.0000], grad_fn=<SelectBackward0>),
   tensor([52.5400, 76.0800, 67.1600,  1.0000], grad_fn=<SelectBackward0>),
   tensor([64.2000, 96.7000, 84.4600,  1.0000], grad_fn=<SelectBackward0>),
   tensor([101.6700, 131.4900, 119.4800,   1.

In [88]:
import torch
from torch.utils.data import Dataset, DataLoader

class JigsawDataset(Dataset):
    def __init__(self, image_files, targets):
        """
        image_files: List of paths to image files
        node_features: List of tensors, each containing node features for an image
        edge_lists: List of tensors, each representing edge index for an image
        targets: List of labels or target data associated with each image
        """
        self.image_files = image_files
        node_features,edge_lists = get_node_features_and_edge_list(self.image_files, grapher_module)
        self.node_features = node_features
        self.edge_lists = edge_lists
        self.targets = targets

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        # Load image here; for simplicity assuming images are already in tensor format
        image = torch.load(self.image_files[idx])
        node_features = self.node_features[idx]
        edge_list = self.edge_lists[idx]
        target = self.targets[idx]
        return image, node_features, edge_list, target


In [110]:
print(input_data[1].shape)

(120, 120, 3)


In [119]:
X_train, y_train = input_data[1],target_data[1]#train_test_split(input_data[1], target_data[1], test_size=0.2, random_state=42)

train_dataset = JigsawDataset(X_train, y_train)
#test_dataset = JigsawDataset(X_test, y_test)
batch_size =1
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
#test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, drop_last=True)

hi


In [122]:
# Assuming 'train_loader' is your DataLoader instance
for images, node_features, edge_indices, targets in train_loader:
    print("Images shape:", images)
    print("Node features shape:", node_features)
    print("Edge indices shape:", edge_indices)
    print("Targets shape:", targets)



AttributeError: 'numpy.ndarray' object has no attribute 'seek'. You can only torch.load from a file that is seekable. Please pre-load the data into a buffer like io.BytesIO and try to load from it instead.

In [117]:
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import MessagePassing

class JigsawModel(nn.Module):
    def __init__(self):
        super(JigsawModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.gnn = MessagePassing(nn.Conv1d(4, 64, kernel_size=1), aggr='add')
        self.fc1 = nn.Linear(64 * 15 * 15 + 64, 4096)
        self.fc2 = nn.Linear(4096, 1024)
        self.fc3 = nn.Linear(1024, 512)
        self.fc4 = nn.Linear(512, 81)

    def forward(self, img, node_features, edge_index):
        x = self.pool(F.relu(self.conv1(img)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 15 * 15)

        # GNN
        node_features = node_features.permute(1, 0)  # Adjust dimensions
        gnn_output = self.gnn(node_features, edge_index).view(-1, 64)

        x = torch.cat((x, gnn_output), dim=1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x


In [118]:
# Assuming the DataLoader and Dataset are set up with your data
# for i in range(10):
#     dataset = JigsawDataset(images[i], targets)
#     dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

model = JigsawModel()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

def train(model, dataloader, optimizer, criterion):
    model.train()
    total_loss = 0
    for img, node_features, edge_list, target in dataloader:
        optimizer.zero_grad()
        output = model(img, node_features, edge_list)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(dataloader)

# Example training call
for epoch in range(1):  # Number of epochs
    loss = train(model, train_loader, optimizer, criterion)
    print(f"Epoch {epoch}, Loss: {loss}")


TypeError: MessagePassing.__init__() got multiple values for argument 'aggr'